In [469]:
import pandas as pd

In [470]:
cols = [x for x in range(1,26)]

In [471]:
matches_df = pd.read_csv('matches.csv', usecols=cols)

In [472]:
matches_df['opponent'] = matches_df['opponent'].str.split().str[1:].str.join(' ')

In [473]:
matches_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1959 entries, 0 to 1958
Data columns (total 25 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   date          1959 non-null   object 
 1   time          1959 non-null   object 
 2   comp          1959 non-null   object 
 3   round         1959 non-null   object 
 4   day           1959 non-null   object 
 5   venue         1959 non-null   object 
 6   result        1958 non-null   object 
 7   gf            1958 non-null   object 
 8   ga            1958 non-null   object 
 9   opponent      1959 non-null   object 
 10  poss          1049 non-null   float64
 11  attendance    939 non-null    float64
 12  captain       1852 non-null   object 
 13  formation     1843 non-null   object 
 14  referee       1955 non-null   object 
 15  match report  1959 non-null   object 
 16  notes         64 non-null     object 
 17  sh            1050 non-null   float64
 18  sot           1050 non-null 

In [474]:
matches_df['date'] = pd.to_datetime(matches_df['date'])

In [475]:
#clening the gf and ga column and making it an int
matches_df['gf'] = matches_df['gf'].str.split().str[0]
matches_df['gf'] = matches_df['gf'].str.split('.').str[0]

matches_df['ga'] = matches_df['ga'].str.split().str[0]
matches_df['ga'] = matches_df['ga'].str.split('.').str[0]

matches_df.dropna(subset=['gf'], inplace=True)
matches_df['gf'] = matches_df['gf'].astype(int)
matches_df['ga'] = matches_df['ga'].astype(int)

In [476]:
matches_df['opp_code'] = matches_df.opponent.astype('category').cat.codes 
matches_df['team_code'] = matches_df.opponent.astype('category').cat.codes 

In [477]:
matches_df['hour'] = matches_df['time'].str.replace(':.+', '', regex=True).astype(int)

In [478]:
matches_df['round_code'] = matches_df['round'].astype('category').cat.codes 

In [479]:
matches_df['day_code'] = matches_df['date'].dt.dayofweek

In [480]:
#matches_df['target'] = matches_df['result'].astype('category').cat.codes
matches_df['target'] = (matches_df['result'] == 'W').astype(int)

In [481]:
matches_df.head()

,date,time,comp,round,day,venue,result,gf,ga,opponent,...,season,team,xg,xga,opp_code,team_code,hour,round_code,day_code,target
0,2020-10-08,21:30,WCQ,WCQ — CONMEBOL (M),Thu,Away,L,0,1,Argentina,...,2022,Ecuador,NaN,NaN,5,5,21,14,3,0
1,2020-10-13,16:00,WCQ,WCQ — CONMEBOL (M),Tue,Home,W,4,2,Uruguay,...,2022,Ecuador,NaN,NaN,163,163,16,14,1,1
2,2020-11-12,16:00,WCQ,WCQ — CONMEBOL (M),Thu,Away,W,3,2,Bolivia,...,2022,Ecuador,NaN,NaN,18,18,16,14,3,1
3,2020-11-17,16:00,WCQ,WCQ — CONMEBOL (M),Tue,Home,W,6,1,Colombia,...,2022,Ecuador,NaN,NaN,34,34,16,14,1,1
4,2021-06-04,21:30,WCQ,WCQ — CONMEBOL (M),Fri,Away,L,0,2,Brazil,...,2022,Ecuador,NaN,NaN,21,21,21,14,4,0


In [482]:
matches_df.dtypes

date            datetime64[ns]
time                    object
comp                    object
round                   object
day                     object
venue                   object
result                  object
gf                       int64
ga                       int64
opponent                object
poss                   float64
attendance             float64
captain                 object
formation               object
referee                 object
match report            object
notes                   object
sh                     float64
sot                    float64
pk                     float64
pkatt                  float64
season                  object
team                    object
xg                     float64
xga                    float64
opp_code                 int16
team_code                int16
hour                     int64
round_code                int8
day_code                 int64
target                   int64
dtype: object

In [483]:
matches_no_na = matches_df.dropna(subset=['poss','sh', 'sot'])
print('len: ', len(matches_no_na['team'].value_counts()))
matches_no_na['team'].value_counts()


len:  32


Spain             59
England           58
Belgium           58
France            57
Denmark           57
Poland            56
Netherlands       55
Germany           55
Croatia           55
Portugal          53
Wales             51
Serbia            49
Switzerland       49
Brazil            44
Argentina         43
Uruguay           42
Ecuador           36
Senegal           21
Cameroon          18
Tunisia           17
Qatar             15
Morocco           14
Mexico            12
United-States     12
Australia         11
Ghana             10
Japan             10
Canada             9
Costa-Rica         8
IR-Iran            6
Korea-Republic     5
Saudi-Arabia       4
Name: team, dtype: int64

In [484]:
matches_no_na['g/sh'] = matches_no_na['gf'] / matches_no_na['sh']
matches_no_na['g/sot'] = matches_no_na['gf'] / matches_no_na['sot']

In [485]:
matches_no_na['g/sh']  = matches_no_na['g/sh'].fillna(0)
matches_no_na['g/sot']  = matches_no_na['g/sot'].fillna(0)

In [486]:
matches_no_na.drop(list(matches_no_na[matches_no_na['g/sh'] > 1].index), inplace=True)

In [487]:
matches_no_na['g/sh'].isna().sum()

0

In [488]:
from sklearn.ensemble import RandomForestClassifier

In [489]:
rf = RandomForestClassifier(n_estimators=50, min_samples_split=10, random_state=1)

In [490]:
train = matches_no_na[matches_no_na['date'] < '2022-01-01']

In [491]:
test = matches_no_na[matches_no_na['date'] > '2022-01-01']

In [492]:
print(len(train) ,len(test))

910 85


In [493]:
predictors = ['opp_code', 'hour', 'day_code', 'team_code']

In [494]:
rf.fit(train[predictors], train['target'])

RandomForestClassifier(min_samples_split=10, n_estimators=50, random_state=1)

In [495]:
preds = rf.predict(test[predictors])

In [496]:
from sklearn.metrics import accuracy_score

In [497]:
acc = accuracy_score(test['target'], preds)

In [498]:
acc

0.5882352941176471

In [499]:
combined = pd.DataFrame(dict(actual = test['target'], prediction = preds))

In [500]:
pd.crosstab(index=combined['actual'], columns=combined['prediction'])

prediction,0,1
actual,,
0,23,22
1,13,27


In [501]:
from sklearn.metrics import precision_score

In [502]:
precision_score(test['target'], preds)

0.5510204081632653

In [503]:
grouped_matches = matches_no_na.groupby('team')

In [504]:
def rolling_averages(group, cols, new_cols):
    group = group.sort_values('date')
    rolling_stats = group[cols].rolling(3, closed ='left').mean()
    group[new_cols] = rolling_stats
    group = group.dropna(subset=new_cols)
    return group

In [505]:
cols = ['gf', 'ga', 'sh', 'sot', 'poss', 'pk', 'pkatt', 'g/sh', 'g/sot']
new_cols = [f'{c}_rolling' for c in cols]

In [506]:
matches_rolling = matches_no_na.groupby('team').apply(lambda x: rolling_averages(x, cols, new_cols))
matches_rolling.head()

date   time            comp           round  day    venue  \
team                                                                            
Argentina 550 2018-06-30  17:00  FIFA World Cup     Round of 16  Sat  Neutral   
          514 2019-06-15  19:00    Copa América     Group stage  Sat  Neutral   
          515 2019-06-19  21:30    Copa América     Group stage  Wed  Neutral   
          516 2019-06-23  16:00    Copa América     Group stage  Sun  Neutral   
          517 2019-06-28  16:00    Copa América  Quarter-finals  Fri  Neutral   

              result  gf  ga   opponent  ...     g/sot  gf_rolling ga_rolling  \
team                                     ...                                    
Argentina 550      L   3   4     France  ...  0.750000    1.000000   1.666667   
          514      L   0   2   Colombia  ...  0.000000    1.666667   2.666667   
          515      D   1   1   Paraguay  ...  0.500000    1.666667   2.333333   
          516      W   2   0      Qatar  ...  0.250000    1.333333   2.333333   
          517      W   2   0  Venezuela  ...  0.285714    1.000000   1.000000   

              sh_rolling sot_rolling poss_rolling pk_rolling  pkatt_rolling  \
team                                                                          
Argentina 550  15.000000    4.000000    67.000000   0.000000       0.333333   
          514  10.000000    3.333333    61.000000   0.000000       0.000000   
          515  11.000000    4.333333    59.333333   0.000000       0.000000   
          516  10.666667    4.000000    56.000000   0.333333       0.333333   
          517  13.000000    5.333333    54.000000   0.333333       0.333333   

               g/sh_rolling  g/sot_rolling  
team                                        
Argentina 550      0.095679       0.277778  
          514      0.166667       0.472222  
          515      0.166667       0.472222  
          516      0.130952       0.416667  
          517      0.082707       0.250000  

[5 rows x 42 columns]

In [507]:
matches_rolling = matches_rolling.droplevel('team')
matches_rolling.index = range(matches_rolling.shape[0])
matches_rolling

,date,time,comp,round,day,venue,result,gf,ga,opponent,...,g/sot,gf_rolling,ga_rolling,sh_rolling,sot_rolling,poss_rolling,pk_rolling,pkatt_rolling,g/sh_rolling,g/sot_rolling
0,2018-06-30,17:00,FIFA World Cup,Round of 16,Sat,Neutral,L,3,4,France,...,0.750000,1.000000,1.666667,15.000000,4.000000,67.000000,0.000000,0.333333,0.095679,0.277778
1,2019-06-15,19:00,Copa América,Group stage,Sat,Neutral,L,0,2,Colombia,...,0.000000,1.666667,2.666667,10.000000,3.333333,61.000000,0.000000,0.000000,0.166667,0.472222
2,2019-06-19,21:30,Copa América,Group stage,Wed,Neutral,D,1,1,Paraguay,...,0.500000,1.666667,2.333333,11.000000,4.333333,59.333333,0.000000,0.000000,0.166667,0.472222
3,2019-06-23,16:00,Copa América,Group stage,Sun,Neutral,W,2,0,Qatar,...,0.250000,1.333333,2.333333,10.666667,4.000000,56.000000,0.333333,0.333333,0.130952,0.416667
4,2019-06-28,16:00,Copa América,Quarter-finals,Fri,Neutral,W,2,0,Venezuela,...,0.285714,1.000000,1.000000,13.000000,5.333333,54.000000,0.333333,0.333333,0.082707,0.250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
894,2021-11-13,19:45,WCQ,First round,Sat,Home,W,5,1,Belarus,...,0.714286,1.333333,0.666667,14.666667,5.000000,54.666667,0.000000,0.000000,0.089286,0.266667
895,2021-11-13,19:45,WCQ,First round,Sat,Home,W,5,1,Belarus,...,0.714286,2.333333,0.333333,15.333333,5.666667,60.333333,0.333333,0.333333,0.140212,0.371429
896,2021-11-16,19:45,WCQ,First round,Tue,Home,D,1,1,Belgium,...,0.333333,3.666667,0.666667,16.666667,6.333333,63.666667,0.666667,0.666667,0.208995,0.542857
897,2021-11-16,19:45,WCQ,First round,Tue,Home,D,1,1,Belgium,...,0.333333,3.666667,1.000000,14.666667,5.666667,52.666667,0.666667,0.666667,0.226852,0.587302


In [508]:
def make_predictions(data, predictors):
    train = data[data['date'] < '2022-01-01']
    test = data[data['date'] > '2022-01-01']
    rf.fit(train[predictors], train['target'])
    preds = rf.predict(test[predictors])
    combined = pd.DataFrame(dict(actual = test['target'], prediction = preds))
    precision = precision_score(test['target'], preds)
    return combined, precision

In [509]:
combined, precision = make_predictions(matches_rolling, predictors + new_cols)

In [510]:
precision

0.4727272727272727

In [511]:
combined = combined.merge(matches_rolling[['date', 'team', 'opponent', 'result']], left_index=True, right_index=True)

In [512]:
combined

,actual,prediction,date,team,opponent,result
36,1,1,2022-01-27,Argentina,Chile,W
37,1,1,2022-02-01,Argentina,Colombia,W
38,1,1,2022-03-25,Argentina,Venezuela,W
39,0,1,2022-03-29,Argentina,Ecuador,D
132,0,0,2022-01-27,Brazil,Ecuador,D
...,...,...,...,...,...,...
851,1,0,2022-01-27,Uruguay,Paraguay,W
852,1,0,2022-02-01,Uruguay,Venezuela,W
853,1,0,2022-03-24,Uruguay,Peru,W
854,1,1,2022-03-29,Uruguay,Chile,W


In [513]:
merged = combined.merge(combined, left_on=['date', 'team'], right_on=['date', 'opponent'])

In [514]:
merged

,actual_x,prediction_x,date,team_x,opponent_x,result_x,actual_y,prediction_y,team_y,opponent_y,result_y
0,0,1,2022-03-29,Argentina,Ecuador,D,0,0,Ecuador,Argentina,D
1,0,0,2022-01-27,Brazil,Ecuador,D,0,1,Ecuador,Brazil,D
2,0,1,2022-01-27,Ecuador,Brazil,D,0,0,Brazil,Ecuador,D
3,0,0,2022-03-29,Ecuador,Argentina,D,0,1,Argentina,Ecuador,D
4,0,1,2022-01-10,Ghana,Morocco,L,1,1,Morocco,Ghana,W
5,0,1,2022-01-10,Ghana,Morocco,L,1,1,Morocco,Ghana,W
6,0,0,2022-01-10,Ghana,Morocco,L,1,1,Morocco,Ghana,W
7,0,0,2022-01-10,Ghana,Morocco,L,1,1,Morocco,Ghana,W
8,1,1,2022-01-10,Morocco,Ghana,W,0,1,Ghana,Morocco,L
9,1,1,2022-01-10,Morocco,Ghana,W,0,0,Ghana,Morocco,L
